# CHAPTER 3: Face Aging

## Pre-requisites

Import OS functions

In [2]:
import glob
import os
import time

Mounting Google Drive and downloading needed data

In [36]:
# from google.colab import drive

# # Mounting Google Drive
# drive.mount('/drive')
# os.chdir('/drive/My Drive/Colab/GenerativeAdversarialNetworks/Chapter03')

# Load the TensorBoard notebook extension
%load_ext tensorboard

# Download Dataset
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar -P ./data
!tar -xvf ./data/wiki_crop.tar -C ./data

# Clear any logs from previous runs
!rm -rf ./logs/ 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Importing the rest of the needed libraries

In [37]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pickle
from keras import Input, Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Reshape, concatenate, LeakyReLU, Lambda
from tensorflow.keras import backend as K
from keras.layers import Activation, UpSampling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras_preprocessing import image
from scipy.io import loadmat
from tqdm import tqdm

Global variables and OS-specific optimizations

In [38]:

# Fixed problems with Error #15: Initializing libiomp5md.dll, but found libiomp5 already initialized.
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Pickle Object to store variables
PICKLE_FILE_NAME = 'pickle_objects.pkl'

Clear the screen

In [39]:
def clear():
    # for windows
    if os.name == 'nt':
        _ = os.system('cls')
  
    # for mac and linux(here, os.name is 'posix')
    else:
        _ = os.system('clear')

## Build the Encoder, Generator and Discriminator, and the complete Facial Recognition Models

Build the Encoder

In [40]:
def build_encoder():
    """
    Encoder Network
    """
    input_layer = Input(shape=(64, 64, 3))

    # 1st Convolutional Block
    enc = Conv2D(filters=32, kernel_size=5, strides=2, padding='same')(input_layer)
    # enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # 2nd Convolutional Block
    enc = Conv2D(filters=64, kernel_size=5, strides=2, padding='same')(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # 3rd Convolutional Block
    enc = Conv2D(filters=128, kernel_size=5, strides=2, padding='same')(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # 4th Convolutional Block
    enc = Conv2D(filters=256, kernel_size=5, strides=2, padding='same')(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # Flatten layer
    enc = Flatten()(enc)

    # 1st Fully Connected Layer
    enc = Dense(4096)(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # Second Fully Connected Layer
    enc = Dense(100)(enc)

    # Create a model
    model = Model(inputs=[input_layer], outputs=[enc])
    return model


Build the Generator

In [41]:
def build_generator():
    """
    Create a Generator Model with hyperparameters values defined as follows
    """
    latent_dims = 100
    num_classes = 6

    input_z_noise = Input(shape=(latent_dims,))
    input_label = Input(shape=(num_classes,))

    x = concatenate([input_z_noise, input_label])

    x = Dense(2048, input_dim=latent_dims + num_classes)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)

    x = Dense(256 * 8 * 8)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)

    x = Reshape((8, 8, 256))(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters=128, kernel_size=5, padding='same')(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters=64, kernel_size=5, padding='same')(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters=3, kernel_size=5, padding='same')(x)
    x = Activation('tanh')(x)

    model = Model(inputs=[input_z_noise, input_label], outputs=[x])
    return model


Build the Discriminator

In [42]:
def build_discriminator():
    """
    Create a Discriminator Model with hyperparameters values defined as follows
    """
    input_shape = (64, 64, 3)
    label_shape = (6,)
    image_input = Input(shape=input_shape)
    label_input = Input(shape=label_shape)

    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(image_input)
    x = LeakyReLU(alpha=0.2)(x)

    label_input1 = Lambda(expand_label_input)(label_input)
    x = concatenate([x, label_input1], axis=3)

    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[image_input, label_input], outputs=[x])
    return model


Build the Facial Recognition Model

In [43]:
def build_fr_model(input_shape):
    resent_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')
    image_input = resent_model.input
    x = resent_model.layers[-1].output
    out = Dense(128)(x)
    embedder_model = Model(inputs=[image_input], outputs=[out])

    input_layer = Input(shape=input_shape)

    x = embedder_model(input_layer)
    output = Lambda(lambda x: K.l2_normalize(x, axis=-1))(x)

    model = Model(inputs=[input_layer], outputs=[output])
    return model

Build the Combined Face Recognition Network containing the Generator and Encoder

In [44]:
def build_fr_combined_network(encoder, generator, fr_model):
    input_image = Input(shape=(64, 64, 3))
    input_label = Input(shape=(6,))

    latent0 = encoder(input_image)

    gen_images = generator([latent0, input_label])

    fr_model.trainable = False

    resized_images = Lambda(lambda x: K.resize_images(gen_images, height_factor=2, width_factor=2,
                                                      data_format='channels_last'))(gen_images)
    embeddings = fr_model(resized_images)

    model = Model(inputs=[input_image, input_label], outputs=[embeddings])
    return model


## Helper functions

Expand the Label Inputs

In [45]:
def expand_label_input(x):
    x = K.expand_dims(x, axis=1)
    x = K.expand_dims(x, axis=1)
    x = K.tile(x, [1, 32, 32, 1])
    return x

Image Resizer

In [46]:
def build_image_resizer():
    input_layer = Input(shape=(64, 64, 3))

    resized_images = Lambda(lambda x: K.resize_images(x, height_factor=3, width_factor=3,
                                                      data_format='channels_last'))(input_layer)

    model = Model(inputs=[input_layer], outputs=[resized_images])
    return model

Age Calculator

In [47]:
def calculate_age(taken, dob):
    birth = datetime.fromordinal(max(int(dob) - 366, 1))

    if birth.month < 7:
        return taken - birth.year
    else:
        return taken - birth.year - 1

Data Loader

In [48]:
def load_data(wiki_dir, dataset='wiki'):
    # Load the wiki.mat file
    meta = loadmat(os.path.join(wiki_dir, f"{dataset}.mat"))

    # Load the list of all files
    full_path = meta[dataset][0, 0]["full_path"][0]

    # List of Matlab serial date number
    dob = meta[dataset][0, 0]["dob"][0]

    # List of years when photo was taken
    photo_taken = meta[dataset][0, 0]["photo_taken"][0]  # year

    # Calculate age for all dobs
    age = [calculate_age(photo_taken[i], dob[i]) for i in range(len(dob))]

    # Create a list of tuples containing a pair of an image path and age
    images = []
    age_list = []
    for index, image_path in enumerate(full_path):
        images.append(image_path[0])
        age_list.append(age[index])

    # Return a list of all images and respective age
    return images, age_list


Splitting Ages to Specific Categories

In [49]:
def age_to_category(age_list):
    age_list1 = []

    for age in age_list:
        if 0 < age <= 18:
            age_category = 0
        elif 18 < age <= 29:
            age_category = 1
        elif 29 < age <= 39:
            age_category = 2
        elif 39 < age <= 49:
            age_category = 3
        elif 49 < age <= 59:
            age_category = 4
        elif age >= 60:
            age_category = 5

        age_list1.append(age_category)

    return age_list1


Image Loader

In [50]:
def load_images(data_dir, image_paths, image_shape):
    images = None
    number_of_images = len(image_paths)
    pbar = tqdm(total=number_of_images) # Init pbar
    pickle_file_exists = False

    print(f'Loading {number_of_images} images')

    try:
        with open(PICKLE_FILE_NAME, 'rb') as pickle_in:
            # Deserialize class loaded_images
            images = pickle.load(pickle_in)

        pickle_file_exists = True
        print(f'{PICKLE_FILE_NAME} is loaded successfully')
    except FileNotFoundError:
        print(f'{PICKLE_FILE_NAME} does not exist yet. Creating it.')

    while pickle_file_exists == False:
        for i, image_path in enumerate(image_paths):
            pbar.update(n=1) # Increments counter
            try:
                # Load image
                loaded_image = image.load_img(os.path.join(data_dir, image_path), target_size=image_shape)

                # Convert PIL image to numpy ndarray
                loaded_image = image.img_to_array(loaded_image)

                # Add another dimension (Add batch dimension)
                loaded_image = np.expand_dims(loaded_image, axis=0)

                # Concatenate all images into one tensor
                if images is None:
                    images = loaded_image
                else:
                    images = np.concatenate([images, loaded_image], axis=0)
            except Exception as e:
                print(f"Error at {i} with Exception {e}")

    print('Finished loading all images')
    return images


Calculator for Euclidean Distance Loss

In [51]:
def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))


Log Writer

In [52]:
def write_log(callback, name, value, batch_no):
    writer = tf.summary.create_file_writer(callback.log_dir)
    with writer.as_default():
        tf.summary.scalar(name, value, step=batch_no)

RGB image Saver

In [53]:
def save_rgb_img(img, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()

## Putting it All together

### Define hyperparameters


Don't forget to set the boolean values on what to train

In [54]:
data_dir = "data"
wiki_dir = os.path.join(data_dir, "wiki_crop")
epochs = 500
batch_size = 2
image_shape = (64, 64, 3)
z_shape = 100
TRAIN_GAN = True
TRAIN_ENCODER = False
TRAIN_GAN_WITH_FR = False
fr_image_shape = (192, 192, 3)

### Clears the screen

In [55]:
clear()

### Define optimizers

In [56]:
dis_optimizer = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)
gen_optimizer = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)
adversarial_optimizer = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)

### Build and compile the networks

In [57]:
# Build and compile the discriminator network
discriminator = build_discriminator()
discriminator.compile(loss=['binary_crossentropy'], optimizer=dis_optimizer)

# Build and compile the generator network
generator = build_generator()
generator.compile(loss=['binary_crossentropy'], optimizer=gen_optimizer)

# Build and compile the adversarial model
discriminator.trainable = False
input_z_noise = Input(shape=(100,))
input_label = Input(shape=(6,))
recons_images = generator([input_z_noise, input_label])
valid = discriminator([recons_images, input_label])
adversarial_model = Model(inputs=[input_z_noise, input_label], outputs=[valid])
adversarial_model.compile(loss=['binary_crossentropy'], optimizer=gen_optimizer)

### Creating the Tensorboard

In [58]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time.time()))
tensorboard.set_model(generator)
tensorboard.set_model(discriminator)

2021-10-30 20:45:41.198835: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-30 20:45:41.198844: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-30 20:45:41.199298: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


### Load the Dataset

In [59]:
images, age_list = load_data(wiki_dir=wiki_dir, dataset="wiki")
age_cat = age_to_category(age_list)
final_age_cat = np.reshape(np.array(age_cat), [len(age_cat), 1])
classes = len(set(age_cat))
y = to_categorical(final_age_cat, num_classes=len(set(age_cat)))

Load the images.
NOTE: This is the longest running operation here.

In [60]:
loaded_images = load_images(wiki_dir, images, (image_shape[0], image_shape[1]))

Loading 62328 images
pickle_objects.pkl does not exist yet. Creating it.
Error at 3 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/65/10001965_1930-05-23_1961.jpg'
Error at 10 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/99/10004299_1908-08-19_1950.jpg'
Error at 11 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/56/1000456_1933-06-12_1969.jpg'
Error at 12 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/82/10004882_1987-05-16_2010.jpg'
Error at 14 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/61/10005261_1936-06-24_1974.jpg'
Error at 16 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/84/1000684_1972-04-05_2004.jpg'
Error at 17 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/50/10006850_1973-08-17_2010.jpg'
Error at 18 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/77/10007577_1988-06-16_2007.jpg'
Error at 19 with Exception

Error at 298 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/69/20070169_1985-12-28_2013.jpg'
Error at 299 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/94/2007194_1945-06-15_2015.jpg'
Error at 301 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/72/2007372_1951-05-13_2011.jpg'
Error at 304 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/50/200750_1973-04-14_2014.jpg'
Error at 305 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/71/20076571_1981-01-26_2010.jpg'
Error at 306 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/87/2007787_1952-05-06_2014.jpg'
Error at 308 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/97/2008097_1933-05-12_2008.jpg'
Error at 310 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/50/20084050_1986-01-06_2008.jpg'
Error at 311 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/63/20084963_1

Error at 450 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/72/3009272_1986-02-21_2009.jpg'
Error at 452 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/96/30093896_1992-03-14_2013.jpg'
Error at 455 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/63/30099963_1985-10-07_2009.jpg'
Error at 456 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/81/40000581_1990-05-09_2012.jpg'
Error at 459 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/75/40005275_1966-10-05_1993.jpg'
Error at 460 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/62/400062_1939-10-10_1997.jpg'
Error at 462 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/61/4001161_1911-05-20_1984.jpg'
Error at 465 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/93/40014293_1993-04-05_2013.jpg'
Error at 470 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/72/40023572

Error at 554 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/92/500592_1912-04-09_1980.jpg'
Error at 555 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/66/5006566_1979-11-01_2012.jpg'
Error at 556 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/94/500694_1979-09-27_2013.jpg'
Error at 560 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/57/5008457_1987-05-08_2014.jpg'
Error at 563 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/69/5008869_1970-10-16_2011.jpg'
Error at 568 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/64/6000464_1939-01-07_2007.jpg'
Error at 571 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/70/6001570_1911-11-25_1976.jpg'
Error at 572 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/67/600167_1974-11-04_2009.jpg'
Error at 576 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/77/600277_1964-06-

Error at 621 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/79/8003479_1940-02-27_1979.jpg'
Error at 622 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/86/8005486_1980-02-19_2013.jpg'
Error at 623 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/94/800594_1983-09-26_2009.jpg'
Error at 624 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/84/8006084_1960-12-02_2011.jpg'
Error at 625 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/88/8006688_1961-03-14_2007.jpg'
Error at 626 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/64/8006764_1924-06-28_1992.jpg'
Error at 628 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/63/8007563_1975-12-17_2004.jpg'
Error at 629 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/58/8007858_1933-10-19_1964.jpg'
Error at 631 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/72/8008972_1985-

...

Error at 11763 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/89/19104489_1981-01-30_2014.jpg'
Error at 11764 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/69/19104569_1987-11-26_2012.jpg'
Error at 11765 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/85/1910485_1963-08-07_2010.jpg'
Error at 11768 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/86/1910886_1943-02-06_1973.jpg'
Error at 11771 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/59/19109759_1986-08-26_2009.jpg'
Error at 11772 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/98/191098_1949-05-20_2007.jpg'


Error at 11775 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/56/1911156_1979-09-20_2011.jpg'
Error at 11777 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/65/19113965_1989-11-09_2012.jpg'


Error at 11782 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/90/19116390_1987-10-23_2014.jpg'


Error at 11789 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/68/19125768_1958-05-12_2007.jpg'
Error at 11791 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/85/191285_1975-02-04_2008.jpg'
Error at 11793 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/55/19128855_1987-12-27_2013.jpg'
Error at 11797 with Exception [Errno 2] No such file or directory: 'data/wiki_crop/98/19130198_1952-08-06_1997.jpg'


KeyboardInterrupt: 

####Pickling Data for future use

In [ ]:
with open('pickle_objects.pkl', 'wb') as pickle_out:
    # Serialize Loaded Images
    pickle.dump(loaded_images, pickle_out)

### Implement Label Smoothing

In [ ]:
real_labels = np.ones((batch_size, 1), dtype=np.float32) * 0.9
fake_labels = np.zeros((batch_size, 1), dtype=np.float32) * 0.1

### Train GAN

In [ ]:
if TRAIN_GAN:
    print(f'\n\n #################### TRAINING GAN ####################\n\n')
    for epoch in range(epochs):
        print(f"\nEpoch: {epoch + 1} out of {len(range(epochs))}")

        gen_losses = []
        dis_losses = []

        number_of_batches = int(len(loaded_images) / batch_size)
        for index in range(number_of_batches):
            print(f"\tBatch: {index + 1} out of {len(range(number_of_batches))}\n")

            images_batch = loaded_images[index * batch_size:(index + 1) * batch_size]
            images_batch = images_batch / 127.5 - 1.0
            images_batch = images_batch.astype(np.float32)

            y_batch = y[index * batch_size:(index + 1) * batch_size]
            z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))

            """
            Train the discriminator network
            """

            # Generate fake images
            initial_recon_images = generator.predict_on_batch([z_noise, y_batch])

            d_loss_real = discriminator.train_on_batch([images_batch, y_batch], real_labels)
            d_loss_fake = discriminator.train_on_batch([initial_recon_images, y_batch], fake_labels)

            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            print(f"\td_loss:{d_loss}")

            """
            Train the generator network
            """

            z_noise2 = np.random.normal(0, 1, size=(batch_size, z_shape))
            random_labels = np.random.randint(0, 6, batch_size).reshape(-1, 1)
            random_labels = to_categorical(random_labels, 6)

            g_loss = adversarial_model.train_on_batch([z_noise2, random_labels], [1] * batch_size)

            print(f"\tg_loss:{g_loss}")

            gen_losses.append(g_loss)
            dis_losses.append(d_loss)

        # Write losses to Tensorboard
        write_log(tensorboard, 'g_loss', np.mean(gen_losses), epoch)
        write_log(tensorboard, 'd_loss', np.mean(dis_losses), epoch)

        """
        Generate images after every 10th epoch
        """
        if epoch % 10 == 0:
            images_batch = loaded_images[0:batch_size]
            images_batch = images_batch / 127.5 - 1.0
            images_batch = images_batch.astype(np.float32)

            y_batch = y[0:batch_size]
            z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))

            gen_images = generator.predict_on_batch([z_noise, y_batch])

            for i, img in enumerate(gen_images[:5]):
                save_rgb_img(img, path="results/img_{}_{}.png".format(epoch, i))

    # Save networks
    try:
        generator.save_weights("generator.h5")
        discriminator.save_weights("discriminator.h5")
    except Exception as e:
        print(f"Error: {e}")




 #################### TRAINING GAN ####################



Epoch: 1 out of 500
	Batch: 1 out of 31164



2021-10-30 12:23:44.658441: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-30 12:23:44.661739: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


	d_loss:1.4016368389129639


AttributeError: 'int' object has no attribute 'shape'

### Train the Encoder

In [ ]:
if TRAIN_ENCODER:
    # Build and compile encoder
    encoder = build_encoder()
    encoder.compile(loss=euclidean_distance_loss, optimizer='adam')

    # Load the generator network's weights
    try:
        generator.load_weights("generator.h5")
    except Exception as e:
        print(f"Error: {e}")

    z_i = np.random.normal(0, 1, size=(5000, z_shape))

    y = np.random.randint(low=0, high=6, size=(5000,), dtype=np.int64)
    num_classes = len(set(y))
    y = np.reshape(np.array(y), [len(y), 1])
    y = to_categorical(y, num_classes=num_classes)

    print(f'\n\n #################### TRAINING ENCODER ####################\n\n')
    for epoch in range(epochs):
        print(f"\nEpoch: {epoch + 1} out of {len(range(epochs))}")

        encoder_losses = []

        number_of_batches = int(z_i.shape[0] / batch_size)
        for index in range(number_of_batches):
            print(f"\tBatch: {index + 1} out of {len(range(number_of_batches))}\n")

            z_batch = z_i[index * batch_size:(index + 1) * batch_size]
            y_batch = y[index * batch_size:(index + 1) * batch_size]

            generated_images = generator.predict_on_batch([z_batch, y_batch])

            # Train the encoder model
            encoder_loss = encoder.train_on_batch(generated_images, z_batch)
            print(f"\tEncoder loss: {encoder_loss}")

            encoder_losses.append(encoder_loss)

        # Write the encoder loss to Tensorboard
        write_log(tensorboard, "encoder_loss", np.mean(encoder_losses), epoch)

    # Save the encoder model
    encoder.save_weights("encoder.h5")

"""
Optimize the encoder and the generator network
"""


### Train GAN with Facial Recognition

In [ ]:
if TRAIN_GAN_WITH_FR:

    # Load the encoder network
    encoder = build_encoder()
    encoder.load_weights("encoder.h5")

    # Load the generator network
    generator.load_weights("generator.h5")

    image_resizer = build_image_resizer()
    image_resizer.compile(loss=['binary_crossentropy'], optimizer='adam')

    # Face recognition model
    fr_model = build_fr_model(input_shape=fr_image_shape)
    fr_model.compile(loss=['binary_crossentropy'], optimizer="adam")

    # Make the face recognition network as non-trainable
    fr_model.trainable = False

    # Input layers
    input_image = Input(shape=(64, 64, 3))
    input_label = Input(shape=(6,))

    # Use the encoder and the generator network
    latent0 = encoder(input_image)
    gen_images = generator([latent0, input_label])

    # Resize images to the desired shape
    resized_images = Lambda(lambda x: K.resize_images(gen_images, height_factor=3, width_factor=3,
                                                      data_format='channels_last'))(gen_images)
    embeddings = fr_model(resized_images)

    # Create a Keras model and specify the inputs and outputs for the network
    fr_adversarial_model = Model(inputs=[input_image, input_label], outputs=[embeddings])

    # Compile the model
    fr_adversarial_model.compile(loss=euclidean_distance_loss, optimizer=adversarial_optimizer)

    print(f'\n\n #################### TRAINING GAN WITH FACE RECOGNITION ####################\n\n')
    for epoch in range(epochs):
        print(f"\nEpoch: {epoch + 1} out of {len(range(epochs))}")
        reconstruction_losses = []

        number_of_batches = int(len(loaded_images) / batch_size)
        for index in range(number_of_batches):
            print(f"\tBatch: {index + 1} out of {len(range(number_of_batches))}\n")

            images_batch = loaded_images[index * batch_size:(index + 1) * batch_size]
            images_batch = images_batch / 127.5 - 1.0
            images_batch = images_batch.astype(np.float32)

            y_batch = y[index * batch_size:(index + 1) * batch_size]

            images_batch_resized = image_resizer.predict_on_batch(images_batch)

            real_embeddings = fr_model.predict_on_batch(images_batch_resized)

            reconstruction_loss = fr_adversarial_model.train_on_batch([images_batch, y_batch], real_embeddings)

            print(f"\tReconstruction loss: {reconstruction_loss}")

            reconstruction_losses.append(reconstruction_loss)

        # Write the reconstruction loss to Tensorboard
        write_log(tensorboard, "reconstruction_loss", np.mean(reconstruction_losses), epoch)

        """
        Generate images
        """
        if epoch % 10 == 0:
            images_batch = loaded_images[0:batch_size]
            images_batch = images_batch / 127.5 - 1.0
            images_batch = images_batch.astype(np.float32)

            y_batch = y[0:batch_size]
            z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))

            gen_images = generator.predict_on_batch([z_noise, y_batch])

            for i, img in enumerate(gen_images[:5]):
                save_rgb_img(img, path="results/img_opt_{}_{}.png".format(epoch, i))

    # Save improved weights for both of the networks
    generator.save_weights("generator_optimized.h5")
    encoder.save_weights("encoder_optimized.h5")


## Visualizing the Logs

In [ ]:
%tensorboard --logdir=logs